In [1]:
import pandas as pd
from datetime import datetime, date
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

import os
import pandas as pd
import openpyxl as excel
import glob
import cgmquantify as cgm
from statistics import mean

import warnings
warnings.filterwarnings('ignore')

## Data Loading ...

In [2]:
pre_pert = r'C:\Users\arsal\OneDrive\Desktop\OPEN\EPI-GV\PREandPOST PERT Merged and Cleaned Data\pre-pert-cleaned-temp1.csv'
path = r'C:\Users\arsal\OneDrive\Desktop\OPEN\EPI-GV\PREandPOST PERT Merged and Cleaned Data'

df_pre_pert = pd.read_csv(pre_pert, index_col=None, na_values=" null")
df_pre_pert = df_pre_pert.rename(columns={"glucose": "Glucose", "Timestamp": "Time"})
df_pre_pert["Time"] = pd.to_datetime(df_pre_pert["Time"],
                                             format='%d/%m/%Y %H:%M:%S')

#df_pre_pert.drop('Unnamed: 0', inplace=True, axis=1)

df_pre_pert

,Time,Glucose,carbs,Target,Duration
0,2021-11-27 23:55:57,107,NaN,NaN,NaN
1,2021-11-27 23:50:59,135,NaN,NaN,NaN
2,2021-11-27 23:50:58,135,NaN,NaN,NaN
3,2021-11-27 23:45:59,158,NaN,NaN,NaN
4,2021-11-27 23:45:57,158,NaN,NaN,NaN
...,...,...,...,...,...
13812,2021-10-27 13:47:43,106,NaN,NaN,NaN
13813,2021-10-27 13:45:37,99,NaN,NaN,NaN
13814,2021-10-27 13:42:42,109,NaN,NaN,NaN
13815,2021-10-27 13:40:37,106,NaN,NaN,NaN


In [3]:
post_pert = r'C:\Users\arsal\OneDrive\Desktop\OPEN\EPI-GV\PREandPOST PERT Merged and Cleaned Data\post-pert-cleaned-temp1.csv'
path = r'C:\Users\arsal\OneDrive\Desktop\OPEN\EPI-GV\PREandPOST PERT Merged and Cleaned Data'

df_post_pert = pd.read_csv(post_pert, index_col=None, na_values=" null")
df_post_pert = df_post_pert.rename(columns={"glucose": "Glucose", "Timestamp": "Time"})
df_post_pert["Time"] = pd.to_datetime(df_post_pert["Time"],
                                             format='%d/%m/%Y %H:%M:%S')

df_post_pert.drop('Unnamed: 0', inplace=True, axis=1)

df_post_pert

,Time,Glucose,carbs,Target,Duration,fat,protein,Meal Details,Notes,PERT,Lipase,Fodzyme?
0,2022-02-01 23:56:51,109,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2022-02-01 23:51:53,112,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2022-02-01 23:51:51,112,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2022-02-01 23:46:51,112,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2022-02-01 23:41:51,104,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
13955,2022-01-05 00:11:30,324,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
13956,2022-01-05 00:06:29,320,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
13957,2022-01-05 00:06:29,320,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
13958,2022-01-05 00:01:30,323,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## PERT GV Algorithm

In [8]:
def pert_gv(df_pert, verbosity, hours_5minute, postmeal=True):
    """
    df_pert: DataFrame with features including timestamp, glucose, carbs, 
            targets, protein, fats
    verbosity: print the statsa and details
    hours_5minute: CGM points per hour = 12
    postmeal: if true chunk the for data after carb entry else chunk the
             data before carb entries
    """

    i = 0
    df5 = pd.DataFrame()
    df6 = pd.DataFrame()

    all_stats = []
    ranges = [0, 70, 180, 350]  # Define store Glucose ranges

    # lists to store variability metrics after a bad low
    TORless70, TIR, TORmore180, interdaycv, stdd, POR, J_index, LBGI, HBGI, GMI, stats = (
        [] for i in range(11))

    # Assign bin level (ranges) to Glucose values
    df_pert['ranges'] = pd.cut(df_pert['Glucose'], bins=ranges)

    for ind, row in df_pert.iterrows():
        if ((np.isnan(row['carbs']) == False) and (row['carbs'] > 15)):

            if verbosity:
                print(ind, "carbs", row['carbs'])

            if postmeal:
                df4 = df_pert[ind:ind + hours_5minute]
            else:
                df4 = df_pert[ind - hours_5minute:ind]

            if not df5.empty:
                # check for target entry greater than 110
                for ind1, row1 in df4.iterrows():
                    if (np.isnan(row1['Target']) == False):
                        if (row1['Target'] > 110):
                            if verbosity:
                                print("At Index", ind1,
                                      "there is a target entry of",
                                      row1['Target'])
                            end_limit1 = ind1 - ind
                            df5 = df_pert[ind:ind + end_limit1]

                # check for second carb entry with in 30 minutes of first entry
                for ind2, row2 in df4.iloc[1:].iterrows():
                    if (i < 12):
                        if ((np.isnan(row2['carbs']) == False)
                                and (row2['carbs'] > 15)):
                            if verbosity:
                                print("At Index", ind2,
                                      "there is a carb entry of",
                                      row2['carbs'])
                            end_limit2 = ind2 - ind
                            df6 = df_pert[ind:ind + end_limit2]
                    else:
                        break
                    i = i + 1

            if not ((df5.empty) and (df6.empty)):
                if (len(df5) > len(df6)):
                    df4 = df6
                elif (len(df5) < len(df6)):
                    df4 = df5

            if verbosity:
                print(df4)

            dffinal = df4[["Time", "Glucose", "ranges"]]

            dffinal.index = pd.to_datetime(dffinal["Time"],
                                           format='%Y-%m-%d %H:%M:%S')

            # calculate TIR and TOR ranges
            result = (dffinal.groupby(
                [pd.Grouper(key="Time"),
                 "ranges"])["ranges"].count().unstack(0).T.fillna(0))
            
            summed_results = result.sum()

            TORless70.append(
                float(
                    round(summed_results.iloc[0] / summed_results.sum() * 100,
                          2)))

            TIR.append(
                float(
                    round(summed_results.iloc[1] / summed_results.sum() * 100,
                          2)))

            TORmore180.append(
                float(
                    round(summed_results.iloc[2] / summed_results.sum() * 100,
                          2)))

            # Calculate other variability metrics
            stdd.append(dffinal['Glucose'].std())
            interdaycv.append(float(cgm.interdaycv(dffinal)))
            POR.append(float(cgm.POR(dffinal)))
            J_index.append(float(cgm.J_index(dffinal)))
            LBGI.append(float(cgm.LBGI(dffinal)))
            HBGI.append(float(cgm.HBGI(dffinal)))
            GMI.append(float(cgm.GMI(dffinal)))

            stats = [
                stdd, TORless70, TIR, TORmore180, POR, J_index,
                LBGI, HBGI, GMI
            ]

    all_stats = [
        mean(interdaycv),
        mean(stdd),
        mean(TORless70),
        mean(TIR),
        mean(TORmore180),
        mean(POR),
        mean(J_index),
        mean(LBGI),
        mean(HBGI),
        mean(GMI)
    ]

    if verbosity:
        print(all_stats)

    statistics = pd.DataFrame(all_stats).T

    statistics.columns = [
        'Mean CV', 'Mean STD', 'Mean TOR<70 [%]',
        'Mean TIR [%]', 'Mean TOR>180 [%]', 'Mean POR', 'Mean J_index',
        'Mean LGBI', 'Mean HBGI', 'Mean GMI'
    ]

    return statistics

## PERT GV -- Post Meal Analysis

In [9]:
results_path = r'C:\Users\arsal\OneDrive\Desktop\OPEN\EPI-GV\results'
    
statistics_post12 = pert_gv(df_post_pert, 0, 12, postmeal=True)
statistics_post24 = pert_gv(df_post_pert, 0, 24, postmeal=True)
statistics_post36 = pert_gv(df_post_pert, 0, 36, postmeal=True)

statistics_postpert_postmeal = pd.concat(
    [statistics_post12, statistics_post24, statistics_post36], axis=0)

print("Post PERT and Post Meal Statistics")

statistics_postpert_postmeal.to_csv(os.path.join(results_path, "postpert_postmeal.csv"))

statistics_postpert_postmeal

Post PERT and Post Meal Statistics


,Mean CV,Mean STD,Mean TOR<70 [%],Mean TIR [%],Mean TOR>180 [%],Mean POR,Mean J_index,Mean LGBI,Mean HBGI,Mean GMI
0,5.782498,6.054462,6.372588,93.137176,0.490235,35.882353,12.802846,1.696543,0.434840,5.820099
0,9.039827,9.444880,5.441059,94.019765,0.539176,37.107843,13.458106,1.606157,0.406932,5.812114
0,11.482187,12.016976,5.098118,93.758118,1.143765,37.058824,14.132417,1.585585,0.460028,5.818231


In [10]:
statistics_pre12 = pert_gv(df_pre_pert, 0, 12, postmeal=True)
statistics_pre24 = pert_gv(df_pre_pert, 0, 24, postmeal=True)
statistics_pre36 = pert_gv(df_pre_pert, 0, 36, postmeal=True)

statistics_prepert_postmeal = pd.concat(
    [statistics_pre12, statistics_pre24, statistics_pre36], axis=0)

print("Pre PERT and Post Meal Statistics")

statistics_prepert_postmeal.to_csv(os.path.join(results_path, "prepert_postmeal.csv"))

statistics_prepert_postmeal

Pre PERT and Post Meal Statistics


,Mean CV,Mean STD,Mean TOR<70 [%],Mean TIR [%],Mean TOR>180 [%],Mean POR,Mean J_index,Mean LGBI,Mean HBGI,Mean GMI
0,5.837010,6.25015,1.80000,96.80000,1.40000,37.000000,13.017647,1.312286,0.484869,5.838695
0,8.375703,8.96386,1.23336,97.23328,1.53336,36.566667,13.723460,1.171084,0.471637,5.850870
0,10.510777,11.24423,2.04464,96.75544,1.19992,36.911111,14.019874,1.267343,0.421029,5.829471


## PERT GV -- Pre Meal Analysis

In [11]:
statistics_post12 = pert_gv(df_post_pert, 0, 12, postmeal=False)
statistics_post24 = pert_gv(df_post_pert, 0, 24, postmeal=False)
statistics_post36 = pert_gv(df_post_pert, 0, 36, postmeal=False)

statistics_postpert_premeal = pd.concat(
    [statistics_post12, statistics_post24, statistics_post36], axis=0)

print("Post PERT and Pre Meal Statistics")

statistics_postpert_premeal.to_csv(os.path.join(results_path, "postpert_premeal.csv"))

statistics_postpert_premeal

Post PERT and Pre Meal Statistics


,Mean CV,Mean STD,Mean TOR<70 [%],Mean TIR [%],Mean TOR>180 [%],Mean POR,Mean J_index,Mean LGBI,Mean HBGI,Mean GMI
0,5.777872,6.112668,5.098000,93.725529,1.176471,39.215686,12.880851,1.865849,0.487228,5.818528
0,8.057407,8.649678,4.706000,94.117529,1.176471,36.176471,13.766043,1.616246,0.502643,5.850492
0,10.392294,11.177038,3.921765,95.032471,1.045765,35.294118,14.541894,1.442327,0.498939,5.877898


In [12]:
statistics_pre12 = pert_gv(df_pre_pert, 0, 12, postmeal=False)
statistics_pre24 = pert_gv(df_pre_pert, 0, 24, postmeal=False)
statistics_pre36 = pert_gv(df_pre_pert, 0, 36, postmeal=False)

statistics_prepert_premeal = pd.concat(
    [statistics_pre12, statistics_pre24, statistics_pre36], axis=0)

print("Post PERT and Pre Meal Statistics")

statistics_prepert_premeal.to_csv(os.path.join(results_path, "prepert_premeal.csv"))

statistics_prepert_premeal

Post PERT and Pre Meal Statistics


,Mean CV,Mean STD,Mean TOR<70 [%],Mean TIR [%],Mean TOR>180 [%],Mean POR,Mean J_index,Mean LGBI,Mean HBGI,Mean GMI
0,6.491568,7.151907,2.86664,96.13328,1.00008,37.933333,13.727438,1.322433,0.456774,5.894779
0,9.265790,10.195179,3.50008,95.46656,1.03336,36.466667,14.760210,1.302622,0.526210,5.922782
0,11.461332,12.911920,3.17784,95.55552,1.26664,35.488889,15.745037,1.241382,0.631350,5.944256


## PERT-GV -- Breakfast analysis

### POST PERT-GV -- Breakfast analysis

In [13]:
def postpert_gv_bf(df_pert, verbosity, hours_5minute, postmeal=True):
    """
    df_pert: DataFrame with features including timestamp, glucose, carbs, 
            targets, protein, fats
    verbosity: print the statsa and details
    hours_5minute: CGM points per hour = 12
    postmeal: if true chunk the for data after carb entry else chunk the
             data before carb entries
    """

    i = 0
    df5 = pd.DataFrame()
    df6 = pd.DataFrame()

    all_stats = []
    ranges = [0, 54, 70, 180, 350]  # Define store Glucose ranges

    # lists to store variability metrics after a bad low
    TORless70, TIR, TORmore180, interdaycv, stdd, POR, J_index, LBGI, HBGI, GMI, stats = (
        [] for i in range(11))

    # Assign bin level (ranges) to Glucose values
    df_pert['ranges'] = pd.cut(df_pert['Glucose'], bins=ranges)

    for ind, row in df_pert.iterrows():
        if ((np.isnan(row['carbs']) == False)
                and (row['Meal Details'] == "Breakfast")
                and (row['carbs'] > 15)):

            if verbosity:
                print(ind, "carbs", row['carbs'])

            if postmeal:
                df4 = df_pert[ind:ind + hours_5minute]
            else:
                df4 = df_pert[ind - hours_5minute:ind]

            if not df5.empty:
                # check for target entry greater than 110
                for ind1, row1 in df4.iterrows():
                    if (np.isnan(row1['Target']) == False):
                        if (row1['Target'] > 110):
                            
                            if verbosity:
                                print("At Index", ind1,
                                      "there is a target entry of",
                                      row1['Target'])
                                
                            end_limit1 = ind1 - ind
                            df5 = df_pert[ind:ind + end_limit1]

                # check for second carb entry with in 30 minutes of first entry
                for ind2, row2 in df4.iloc[1:].iterrows():
                    if (i < 12):
                        if ((np.isnan(row2['carbs']) == False)
                                and (row2['carbs'] > 15)):
                            if verbosity:
                                print("At Index", ind2,
                                      "there is a carb entry of",
                                      row2['carbs'])
                            end_limit2 = ind2 - ind
                            df6 = df_pert[ind:ind + end_limit2]
                    else:
                        break
                    i = i + 1

            if not ((df5.empty) and (df6.empty)):
                if (len(df5) > len(df6)):
                    df4 = df6
                elif (len(df5) < len(df6)):
                    df4 = df5

            if verbosity:
                print(df4)

            dffinal = df4[["Time", "Glucose", "ranges"]]

            dffinal.index = pd.to_datetime(dffinal["Time"],
                                           format='%Y-%m-%d %H:%M:%S')

            # calculate TIR and TOR ranges
            result = (dffinal.groupby(
                [pd.Grouper(key="Time"),
                 "ranges"])["ranges"].count().unstack(0).T.fillna(0))
            summed_results = result.sum()

            TORless70.append(
                float(
                    round(summed_results.iloc[0] / summed_results.sum() * 100,
                          2)))

            TIR.append(
                float(
                    round(summed_results.iloc[1] / summed_results.sum() * 100,
                          2)))

            TORmore180.append(
                float(
                    round(summed_results.iloc[2] / summed_results.sum() * 100,
                          2)))

            # Calculate other variability metrics
            stdd.append(dffinal['Glucose'].std())
            interdaycv.append(float(cgm.interdaycv(dffinal)))
            POR.append(float(cgm.POR(dffinal)))
            J_index.append(float(cgm.J_index(dffinal)))
            LBGI.append(float(cgm.LBGI(dffinal)))
            HBGI.append(float(cgm.HBGI(dffinal)))
            GMI.append(float(cgm.GMI(dffinal)))

            stats = [
                stdd, TORless70, TIR, TORmore180, POR, J_index, LBGI, HBGI, GMI
            ]

    all_stats = [
        mean(interdaycv),
        mean(stdd),
        mean(TORless70),
        mean(TIR),
        mean(TORmore180),
        mean(POR),
        mean(J_index),
        mean(LBGI),
        mean(HBGI),
        mean(GMI)
    ]

    if verbosity:
        print(all_stats)

    statistics = pd.DataFrame(all_stats).T

    statistics.columns = [
        'Mean CV', 'Mean STD', 'Mean TOR<70 [%]', 'Mean TIR [%]',
        'Mean TOR>180 [%]', 'Mean POR', 'Mean J_index', 'Mean LGBI',
        'Mean HBGI', 'Mean GMI'
    ]

    return statistics

In [14]:
results_path = r'C:\Users\arsal\OneDrive\Desktop\OPEN\EPI-GV\results'
    
statistics_post12 = postpert_gv_bf(df_post_pert, 0, 12, postmeal=True)
statistics_post24 = postpert_gv_bf(df_post_pert, 0, 24, postmeal=True)
statistics_post36 = postpert_gv_bf(df_post_pert, 0, 36, postmeal=True)

statistics_postpert_postmeal_bf = pd.concat(
    [statistics_post12, statistics_post24, statistics_post36], axis=0)

print("Post PERT and Post Breakfast Statistics")

statistics_postpert_postmeal_bf.to_csv(os.path.join(results_path, "postpert_postbreakfast.csv"))

statistics_postpert_postmeal_bf

Post PERT and Post Breakfast Statistics


,Mean CV,Mean STD,Mean TOR<70 [%],Mean TIR [%],Mean TOR>180 [%],Mean POR,Mean J_index,Mean LGBI,Mean HBGI,Mean GMI
0,6.854578,6.379354,3.868929,10.118929,86.011786,31.250000,10.759998,3.142632,0.251940,5.587597
0,9.212882,8.851399,1.934643,9.077143,88.244643,37.946429,11.804911,2.662018,0.358883,5.638676
0,12.572647,12.559472,1.289643,7.441429,89.582857,36.309524,13.421470,2.315662,0.531928,5.716167


In [15]:
results_path = r'C:\Users\arsal\OneDrive\Desktop\OPEN\EPI-GV\results'
    
statistics_post12 = postpert_gv_bf(df_post_pert, 0, 12, postmeal=False)
statistics_post24 = postpert_gv_bf(df_post_pert, 0, 24, postmeal=False)
statistics_post36 = postpert_gv_bf(df_post_pert, 0, 36, postmeal=False)

statistics_postpert_premeal_bf = pd.concat(
    [statistics_post12, statistics_post24, statistics_post36], axis=0)

print("Post PERT and Pre Breakfast Statistics")

statistics_postpert_premeal_bf.to_csv(os.path.join(results_path, "postpert_prebreakfast.csv"))

statistics_postpert_premeal_bf

Post PERT and Pre Breakfast Statistics


,Mean CV,Mean STD,Mean TOR<70 [%],Mean TIR [%],Mean TOR>180 [%],Mean POR,Mean J_index,Mean LGBI,Mean HBGI,Mean GMI
0,6.084285,5.816565,4.166786,5.952143,86.607143,41.964286,11.332513,3.351959,0.598089,5.625897
0,8.605109,8.513245,3.571429,6.398929,86.607143,36.160714,12.540107,2.777364,0.613839,5.695700
0,11.191334,11.190382,2.381071,5.158929,89.384643,35.615079,13.418784,2.329765,0.565684,5.741416


### PRE PERT-GV -- Breakfast analysis

In [16]:
def prepert_gv_bf(df_pert, verbosity, hours_5minute, postmeal=True):
    """
    df_pert: DataFrame with features including timestamp, glucose, carbs, 
            targets, protein, fats
    verbosity: print the statsa and details
    hours_5minute: CGM points per hour = 12
    postmeal: if true chunk the for data after carb entry else chunk the
             data before carb entries
    """

    i = 0
    df5 = pd.DataFrame()
    df6 = pd.DataFrame()

    all_stats = []
    ranges = [0, 70, 180, 350]  # Define store Glucose ranges

    # lists to store variability metrics after a bad low
    TORless54, TORless70, TIR, TORmore180, interdaycv, stdd, POR, J_index, LBGI, HBGI, GMI, stats = (
        [] for i in range(12))

    # Assign bin level (ranges) to Glucose values
    df_pert['ranges'] = pd.cut(df_pert['Glucose'], bins=ranges)
    
    df_pert['Time'] = pd.to_datetime(df_pert['Time'],
                                           format='%Y-%m-%d %H:%M:%S')
    
    df_pert['hour'] = df_pert['Time'].dt.hour
    
    for ind, row in df_pert.iterrows():
        if ((np.isnan(row['carbs']) == False)
                and (row['hour'] <= 12)
                and (row['carbs'] == 50)):

            if verbosity:
                print(ind, "carbs", row['carbs'])

            if postmeal:
                df4 = df_pert[ind:ind + hours_5minute]
            else:
                df4 = df_pert[ind - hours_5minute:ind]

            if not df5.empty:
                # check for target entry greater than 110
                for ind1, row1 in df4.iterrows():
                    if (np.isnan(row1['Target']) == False):
                        if (row1['Target'] > 110):
                            
                            if verbosity:
                                print("At Index", ind1,
                                      "there is a target entry of",
                                      row1['Target'])
                                
                            end_limit1 = ind1 - ind
                            df5 = df_pert[ind:ind + end_limit1]

                # check for second carb entry with in 30 minutes of first entry
                for ind2, row2 in df4.iloc[1:].iterrows():
                    if (i < 12):
                        if ((np.isnan(row2['carbs']) == False)
                                and (row2['carbs'] > 15)):
                            if verbosity:
                                print("At Index", ind2,
                                      "there is a carb entry of",
                                      row2['carbs'])
                            end_limit2 = ind2 - ind
                            df6 = df_pert[ind:ind + end_limit2]
                    else:
                        break
                    i = i + 1

            if not ((df5.empty) and (df6.empty)):
                if (len(df5) > len(df6)):
                    df4 = df6
                elif (len(df5) < len(df6)):
                    df4 = df5

            if verbosity:
                print(df4)

            dffinal = df4[["Time", "Glucose", "ranges"]]

            dffinal.index = pd.to_datetime(dffinal["Time"],
                                           format='%Y-%m-%d %H:%M:%S')

            # calculate TIR and TOR ranges
            result = (dffinal.groupby(
                [pd.Grouper(key="Time"),
                 "ranges"])["ranges"].count().unstack(0).T.fillna(0))
            summed_results = result.sum()
            
            TORless70.append(
                float(
                    round(summed_results.iloc[0] / summed_results.sum() * 100,
                          2)))

            TIR.append(
                float(
                    round(summed_results.iloc[1] / summed_results.sum() * 100,
                          2)))

            TORmore180.append(
                float(
                    round(summed_results.iloc[2] / summed_results.sum() * 100,
                          2)))

            # Calculate other variability metrics
            stdd.append(dffinal['Glucose'].std())
            interdaycv.append(float(cgm.interdaycv(dffinal)))
            POR.append(float(cgm.POR(dffinal)))
            J_index.append(float(cgm.J_index(dffinal)))
            LBGI.append(float(cgm.LBGI(dffinal)))
            HBGI.append(float(cgm.HBGI(dffinal)))
            GMI.append(float(cgm.GMI(dffinal)))

            stats = [
                stdd, TORless70, TIR, TORmore180, POR, J_index, LBGI, HBGI, GMI
            ]

    all_stats = [
        mean(interdaycv),
        mean(stdd),
        mean(TORless70),
        mean(TIR),
        mean(TORmore180),
        mean(POR),
        mean(J_index),
        mean(LBGI),
        mean(HBGI),
        mean(GMI)
    ]

    if verbosity:
        print(all_stats)

    statistics = pd.DataFrame(all_stats).T

    statistics.columns = [
        'Mean CV', 'Mean STD', 'Mean TOR<70 [%]', 'Mean TIR [%]',
        'Mean TOR>180 [%]', 'Mean POR', 'Mean J_index', 'Mean LGBI',
        'Mean HBGI', 'Mean GMI'
    ]

    return statistics

In [17]:
results_path = r'C:\Users\arsal\OneDrive\Desktop\OPEN\EPI-GV\results'
    
statistics_post12 = prepert_gv_bf(df_pre_pert, 0, 12, postmeal=True)
statistics_post24 = prepert_gv_bf(df_pre_pert, 0, 24, postmeal=True)
statistics_post36 = prepert_gv_bf(df_pre_pert, 0, 36, postmeal=True)

statistics_prepert_postmeal_bf = pd.concat(
    [statistics_post12, statistics_post24, statistics_post36], axis=0)

print("Pre PERT and Post Breakfast Statistics")

statistics_prepert_postmeal_bf.to_csv(os.path.join(results_path, "prepert_postbreakfast.csv"))

statistics_prepert_postmeal_bf

Pre PERT and Post Breakfast Statistics


,Mean CV,Mean STD,Mean TOR<70 [%],Mean TIR [%],Mean TOR>180 [%],Mean POR,Mean J_index,Mean LGBI,Mean HBGI,Mean GMI
0,5.790685,6.023297,5.247037,89.505926,5.247037,32.098765,13.520068,2.362084,1.137840,5.825587
0,9.511592,10.078262,3.086296,91.666667,5.247037,36.419753,14.835803,1.753188,1.090802,5.882470
0,12.736019,13.816444,5.864074,90.226667,3.909259,36.316872,15.684899,1.933433,0.979019,5.882360


In [18]:
results_path = r'C:\Users\arsal\OneDrive\Desktop\OPEN\EPI-GV\results'
    
statistics_post12 = prepert_gv_bf(df_pre_pert, 0, 12, postmeal=False)
statistics_post24 = prepert_gv_bf(df_pre_pert, 0, 24, postmeal=False)
statistics_post36 = prepert_gv_bf(df_pre_pert, 0, 36, postmeal=False)

statistics_prepert_premeal_bf = pd.concat(
    [statistics_post12, statistics_post24, statistics_post36], axis=0)

print("Pre PERT and Pre Breakfast Statistics")

statistics_prepert_premeal_bf.to_csv(os.path.join(results_path, "prepert_prebreakfast.csv"))

statistics_prepert_premeal_bf

Pre PERT and Pre Breakfast Statistics


,Mean CV,Mean STD,Mean TOR<70 [%],Mean TIR [%],Mean TOR>180 [%],Mean POR,Mean J_index,Mean LGBI,Mean HBGI,Mean GMI
0,7.920982,7.881035,9.876296,88.580370,1.543333,38.888889,13.291283,2.797564,0.623650,5.800559
0,10.695311,10.632209,9.414074,89.814444,0.771481,40.432099,13.597952,2.561430,0.458497,5.788082
0,12.748168,12.834875,7.304444,92.181111,0.514444,36.111111,13.699480,2.239896,0.342236,5.761948
